# Day 14. Task 05
# Пайплайны и ООП

## 0. Импорты

## 1. Пайплайн препроцессинга

Создайте три кастомных трансформера согласно указаниям ниже. Первые два трансформера из списка будут использованы внутри [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html).

1. Класс `FeatureExtractor()`:
    - принимает на вход датафрейм с полями `uid`, `labname`, `numTrials`, `timestamp` из файла `datasets/checker_submits.csv`
    - вытаскивает `hour` из `timestamp`
    - вытаскивает `weekday` из `timestamp` (сохраняет в виде цифр)
    - удаляет столбец `timestamp`
    - возвращает новый датафрейм
    
2. Класс `MyOneHotEncoder()`:
    - принимает на вход датафрейм из предыдущего трансформера и название столбца с целевой переменной
    - обнаруживает все категориальные признаки и трансормирует их при помощи `OneHotEncoder()`. Если целевая переменная является тоже категориальной, то трансформация на нее не распространяется
    - удаляет изначальные категориальные столбцы из датафрейма
    - возвращает датафрейм с признаками и series со столбцом целевой переменной
    
3. Класс `TrainValidationTest()`:
    - принимает на вход `X` и `y`
    - возвращает `X_train`, `X_valid`, `X_test`, `y_train`, `y_valid`, `y_test` (`test_size=0.2`, `random_state=21`, `stratified`).

## 2. Пайплайн выбора модели

Класс `ModelSelection()`
   - Принимает на вход список экземпляров `GridSearchCV` и словарь, в котором ключами являются индексы из этого списка, а значениями – названия моделей. Ниже пример в обратном порядке (для лучшего понимания, строки кода не будут работать при прогоне). Такой порядок позволяет посмотреть на это от общего к частному:
    
    
```
ModelSelection(grids, grid_dict)

grids = [gs_svm, gs_tree, gs_rf]

gs_svm = GridSearchCV(estimator=svm, param_grid=svm_params, scoring='accuracy', cv=2, n_jobs=jobs), where jobs you can specify by yourself

svm_params = [{'kernel':('linear', 'rbf', 'sigmoid'), 'C':[0.01, 0.1, 1, 1.5, 5, 10], 'gamma': ['scale', 'auto'], 'class_weight':('balanced', None), 'random_state':[21], 'probability':[True]}]
```

   - метод `choose()` принимает на вход `X_train`, `y_train`, `X_valid`, `y_valid` и возвращает название лучшего классификатора среди всех моделей на валидационной выборке
   - метод `best_results()` возвращает датафрейм со столбцами `model`, `params`, `valid_score`, где строки – это модели, являющиеся лучшими в своем классе моделей (пример ниже, ваши значения будут другими)
    
```
model	params	valid_score
0	SVM	{'C': 10, 'class_weight': None, 'gamma': 'auto...	0.772727
1	Decision Tree	{'class_weight': 'balanced', 'criterion': 'gin...	0.801484
2	Random Forest	{'class_weight': None, 'criterion': 'entropy',...	0.855288
```

   - когда вы итерируетесь по параметрам класса моделей, выводите на экран название этого класса, а также показывайте прогресс, используя `tqdm.notebook`, в конце цикла этого класса моделей выведете на экран лучшую модель этого класса (ниже пример, значения могут отличаться)
    
```
Estimator: SVM
100%
72/72 [01:32<00:00, 1.36it/s]
Best params: {'C': 10, 'class_weight': None, 'gamma': 'auto', 'kernel': 'rbf', 'probability': True, 'random_state': 21}
Best training accuracy: 0.773
Validation set accuracy score for best params: 0.878 

Estimator: Decision Tree
100%
196/196 [01:07<00:00, 1.22it/s]
Best params: {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 21, 'random_state': 21}
Best training accuracy: 0.801
Validation set accuracy score for best params: 0.867 

Estimator: Random Forest
100%
784/784 [06:47<00:00, 1.13s/it]
Best params: {'class_weight': None, 'criterion': 'entropy', 'max_depth': 22, 'n_estimators': 50, 'random_state': 21}
Best training accuracy: 0.855
Validation set accuracy score for best params: 0.907 

Classifier with best validation set accuracy: Random Forest
```

## 3. Финализация

Класс `Finalize()`
   - принимает на вход estimator (класс, который оценивает качество модели)
   - у класса должен быть метод `save_model()`, который сохраняет финальную модель с заданным путем и именем и сообщает, что модель успешно сохранена, а также метод `final_score()`, который принимает на вход `X_train`, `y_train`, `X_test`, `y_test` и возвращает accuracy модели, как в примере ниже:

```
final.final_score(X_train, y_train, X_test, y_test)
Accuracy of the final model is 0.908284023668639
```

## 4. Основная программа

1. Загрузите данные из файла.
2. Создайте пайплайн препроцессинга, который состоит из двух написанных вами трансформеров: `FeatureExtractor()` и `MyOneHotEncoder()`.
```
preprocessing = Pipeline([('feature_extractor', FeatureExtractor()), ('onehot_encoder', MyOneHotEncoder('dayofweek'))])
```
3. Примените этот пайплайн и его метод `fit_transform()` по отношению к изначальному датасету.
```
data = preprocessing.fit_transform(df)
```
4. Получите `X_train`, `X_valid`, `X_test`, `y_train`, `y_valid`, `y_test`, используя `TrainValidationTest()` и результат предыдущего пайплайна.
5. Создайте объект класса `ModelSelection()`, воспользуйтесь методом `choose()`, применив его к тем моделям и тем параметрам моделей, которые вы хотите. Получите датафрейм с лучшими результатами.
6. Создайте объект класса `Finalize()` с вашей лучшей моделью. Воспользуйтесь методом `final_score()` и сохраните модель в формате `name_of_the_model_{accuracy on test dataset}.sav`.

Это всё. Поздравляем! Это серьезный результат. Вы смогли автоматизировать свою работу.